In [1]:
%matplotlib inline

In [2]:
import os
import sys

In [3]:
lmu_path = os.path.abspath("../lmu")
sys.path.append(lmu_path)


In [9]:
from lmu import LMUCell

from keras.applications import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Input, GlobalAveragePooling2D, Conv2DTranspose, Reshape
from keras.layers import TimeDistributed
from keras.layers.recurrent import RNN
from keras.models import Sequential, Model
from keras.initializers import Constant
from keras.utils import multi_gpu_model, to_categorical

In [5]:
def lmu_layer(return_sequences=False,**kwargs):
    return RNN(LMUCell(units=6,
                       order=6,
                       theta=30,
                       input_encoders_initializer=Constant(1),
                       hidden_encoders_initializer=Constant(0),
                       memory_encoders_initializer=Constant(0),
                       input_kernel_initializer=Constant(0),
                       hidden_kernel_initializer=Constant(0),
                       memory_kernel_initializer='glorot_normal',
                      ),
               return_sequences=return_sequences,
               **kwargs)

In [33]:
seq_len = 300

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in resnet_model.layers:
    layer.trainable=False
    
x = resnet_model.output
base_out = GlobalAveragePooling2D()(x)
base_model = Model(inputs=resnet_model.input, outputs=base_out)

input_layer = Input(shape=(seq_len, 224, 224, 3))
x = TimeDistributed(base_model)(input_layer)
# x = Reshape(target_shape=(seq_len, 2048))(x)
x = lmu_layer(return_sequences=False)(x)
out = Dense(1960)(x)

v = Dense(2)(out)

deconv_shape = (None, 14, 14, 10)
heat = Reshape(deconv_shape[1:])(out)
heat = Conv2DTranspose(filters=10,kernel_size=3,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=5,kernel_size=3,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=2,kernel_size=3,strides=2,activation='relu', padding = 'same')(heat)
heat = Conv2DTranspose(filters=1,kernel_size=3,strides=2,activation='relu', padding = 'same')(heat)
model = Model(inputs=input_layer, outputs=[heat,v])

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [34]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 300, 224, 224 0                                            
__________________________________________________________________________________________________
time_distributed_17 (TimeDistri (None, 300, 2048)    23587712    input_34[0][0]                   
__________________________________________________________________________________________________
rnn_17 (RNN)                    (None, 6)            14462       time_distributed_17[0][0]        
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 1960)         13720       rnn_17[0][0]                     
__________________________________________________________________________________________________
reshape_16